# RLSS2023 - DQN Tutorial: Fitted Q Iteration (FQI)

Website: https://rlsummerschool.com/

Github repository: https://github.com/araffin/rlss23-dqn/

Gymnasium documentation: https://gymnasium.farama.org/

## Introduction

In this notebook, you will implement the Fitted Q Iteration( (FQI) algorithm to solve the [CartPole](https://gymnasium.farama.org/environments/classic_control/cart_pole/) problem.
This notebooks will first cover the basics for using the Gymnasium library: how to instantiate an environment, step into it and collect training data from the FQI algorithm.

You will then learn how to implement step-by-step the FQI algorithm which is the predecessor of the [Deep Q-Network (DQN)](https://stable-baselines3.readthedocs.io/en/master/modules/dqn.html) algorithm.

In [ ]:
# for autoformatting
%load_ext jupyter_black

## Install Dependencies

In [ ]:
!pip install git+https://github.com/araffin/rlss23-dqn/ --upgrade

## First steps with the Gym interface

An environment that follows the [gym interface](https://gymnasium.farama.org/) is quite simple to use.
It provides to this user mainly three methods, which have the following signature (for gym versions > 0.26):

- `reset()` called at the beginning of an episode, it returns an observation and a dictionary with additional info (defaults to an empty dict)
- `step(action)` called to take an action with the environment, it returns the next observation, the immediate reward, whether new state is a terminal state (episode is finished), whether the max number of timesteps is reached (episode is artificially finished), and additional information
- (Optional) `render()` which allow to visualize the agent in action. Note that graphical interface does not work on google colab, so we cannot use it directly (we have to rely on `render_mode='rbg_array'` to retrieve an image of the scene).

Under the hood, it also contains two useful properties:
- `observation_space` which one of the gym spaces (`Discrete`, `Box`, ...) and describe the type and shape of the observation
- `action_space` which is also a gym space object that describes the action space, so the type of action that can be taken

The best way to learn about [gym spaces](https://gymnasium.farama.org/api/spaces/) is to look at the [source code](https://github.com/Farama-Foundation/Gymnasium/tree/main/gymnasium/spaces), but you need to know at least the main ones:
- `gym.spaces.Box`: A (possibly unbounded) box in $R^n$. Specifically, a Box represents the Cartesian product of n closed intervals. Each interval has the form of one of [a, b], (-oo, b], [a, oo), or (-oo, oo). Example: A 1D-Vector or an image observation can be described with the Box space.
```python
# Example for using image as input:
observation_space = spaces.Box(low=0, high=255, shape=(HEIGHT, WIDTH, N_CHANNELS), dtype=np.uint8)
```                                       

- `gym.spaces.Discrete`: A discrete space in $\{ 0, 1, \dots, n-1 \}$
  Example: if you have two actions ("left" and "right") you can represent your action space using `Discrete(2)`, the first action will be 0 and the second 1.

## CartPole Environment

For this example, we will use CartPole environment, a classic control problem.

"A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The system is controlled by applying a force of +1 or -1 to the cart. The pendulum starts upright, and the goal is to prevent it from falling over. A reward of +1 is provided for every timestep that the pole remains upright. "

Cartpole environment: [https://gymnasium.farama.org/environments/classic_control/cart_pole/](https://gymnasium.farama.org/environments/classic_control/cart_pole/)

![Cartpole](https://cdn-images-1.medium.com/max/1143/1*h4WTQNVIsvMXJTCpXm_TAw.gif)

In [ ]:
import gymnasium as gym

# Instantiate the environment
env = gym.make("CartPole-v1")

In [ ]:
# Box(4,) means that it is a Vector with 4 components
print("Observation space:", env.observation_space)
print("Shape:", env.observation_space.shape)
# Discrete(2) means that there is two discrete actions
print("Action space:", env.action_space)

In [ ]:
# The reset method is called at the beginning of an episode
obs, info = env.reset()

In [ ]:
# Sample a random action
action = env.action_space.sample()
print(f"Sampled action: {action}")

In [ ]:
# step in the environment
obs, reward, terminated, truncated, info = env.step(action)

In [ ]:
# Note the obs is a numpy array
# info is an empty dict for now but can contain any debugging info
# reward is a scalar
print(obs.shape, reward, terminated, truncated, info)

### Exercise (10 minutes): write the function to collect data

This function collects an offline dataset of transitions that will be used to train a model using the FQI algorithm.

See docstring of the function for what is expected as input/output.

In [ ]:
from dataclasses import dataclass

import numpy as np
from gymnasium import spaces


@dataclass
class OfflineData:
    """
    A class to store transitions.
    """

    observations: np.ndarray
    next_observations: np.ndarray
    actions: np.ndarray
    rewards: np.ndarray
    terminateds: np.ndarray

In [ ]:
def collect_data(env_id: str, n_steps: int = 50_000) -> OfflineData:
    """
    Collect transitions using a random agent (sample action randomly).

    :param env_id: The name of the environment.
    :param n_steps: Number of steps to perform in the env.
    :return: The collected transitions.
    """
    # Create the Gym env
    env = gym.make(env_id)

    assert isinstance(env.observation_space, spaces.Box)
    # Numpy arrays (buffers) to collect the data
    observations = np.zeros((n_steps, *env.observation_space.shape))
    next_observations = np.zeros((n_steps, *env.observation_space.shape))
    # Discrete actions
    actions = np.zeros((n_steps, 1))
    rewards = np.zeros((n_steps,))
    terminateds = np.zeros((n_steps,))

    # Variable to know if the episode is over (done = terminated or truncated)
    done = False
    # Start the first episode
    obs, _ = env.reset()

    ### YOUR CODE HERE
    # TODO: Collect transitions for `n_steps` using
    # a random agent (sample action uniformly)
    # Do not forget to reset the environment if the current episode is over
    # (done = terminated or truncated)
    #
    # TODO:
    # 1. Sample a random action
    # 2. Step in the env using this random action
    # 3. Retrieve the new transition data (observation, reward, ...)
    #  and update the numpy arrays (buffers)
    # 4. Repeat until you collected `n_steps` transitions

    for idx in range(n_steps):
        # Sample a random action
        action = env.action_space.sample()
        # Step in the environment
        next_obs, reward, terminated, truncated, info_ = env.step(action)

        # Store the transition
        observations[idx, :] = obs
        next_observations[idx, :] = next_obs
        actions[idx, :] = action
        rewards[idx] = reward
        # Only record true termination (timeouts/truncations are artificial terminations)
        terminateds[idx] = terminated
        # Update current observation
        obs = next_obs
        # Check if the episode is over
        done = terminated or truncated

        # Don't forget to reset the env at the end of an episode
        if done:
            obs, _ = env.reset()

    ### END OF YOUR CODE

    return OfflineData(
        observations,
        next_observations,
        actions,
        rewards,
        terminateds,
    )

Let's try the collect data method:

In [ ]:
env_id = "CartPole-v1"
n_steps = 50_000
# Collect transitions for n_steps
data = collect_data(env_id=env_id, n_steps=n_steps)

In [ ]:
# Check the length of the collected data
assert len(data.observations) == n_steps
assert len(data.actions) == n_steps
# Check that there are multiple episodes in the data
assert not np.all(data.terminateds)
assert np.any(data.terminateds)
# Check the shape of the collected data
if env_id == "CartPole-v1":
    assert data.observations.shape == (n_steps, 4)
    assert data.next_observations.shape == (n_steps, 4)
assert data.actions.shape == (n_steps, 1)
assert data.rewards.shape == (n_steps,)

In [ ]:
from pathlib import Path

from dqn_tutorial.fqi import save_data

output_filename = Path("../data") / f"{env_id}_data"
# Create folder if it doesn't exist
output_filename.parent.mkdir(parents=True, exist_ok=True)

# Save collected data using numpy
save_data(data, output_filename)

## Fitted Q Iteration algorithm



In [ ]:
from functools import partial
from pathlib import Path
from typing import Optional

import gymnasium as gym
import numpy as np
from gymnasium import spaces
from sklearn import tree
from sklearn.base import RegressorMixin
from sklearn.exceptions import NotFittedError
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
def create_model_input(
    obs: np.ndarray,
    actions: np.ndarray,
    features_extractor: Optional[PolynomialFeatures] = None,
) -> np.ndarray:
    """
    Concatenate observation (batch_size, n_features)
    and actions (batch_size, 1) along the feature axis.

    :param obs: A batch of observations.
    :param actions: A batch of actions.
    :param features_extractor: Optionally a preprocessor
        to extract features like PolynomialFeatures.
    :return: The input for the scikit-learn model
        (batch_size, n_features + 1)
    """
    # Concatenate the observations and actions
    # so we can predict qf(s_t, a_t)
    model_input = np.concatenate((obs, actions), axis=1)
    # Optionally: extract features from the input using preprocessor
    if features_extractor is not None:
        try:
            model_input = features_extractor.transform(model_input)
        except NotFittedError:
            # First interation: fit the features_extractor
            model_input = features_extractor.fit_transform(model_input)
    return model_input

In [ ]:
# First choose the regressor
model_class = LinearRegression  # tree.DecisionTreeRegressor, RandomForestRegressor, ...
# Optionally: extract features before feeding the input to the model
features_extractor = PolynomialFeatures(degree=2)
# features_extractor = None

In [ ]:
from dqn_tutorial.fqi import load_data

env_id = "CartPole-v1"
output_filename = Path("../data") / f"{env_id}_data.npz"
render_mode = "rgb_array"

# Create test environment
env = gym.make(env_id, render_mode=render_mode)

# Load saved transitions
data = load_data(output_filename)

In [ ]:
# First iteration:
# The target q-value is the reward obtained
targets = data.rewards.copy()
# Create input for current observations
current_obs_input = create_model_input(data.observations, data.actions, features_extractor)
# Fit the estimator for the current target
model = model_class().fit(current_obs_input, targets)

### 1. Exercise (10 minutes): write the function to predict Q-Values

In [ ]:
def get_q_values(
    model: RegressorMixin,
    obs: np.ndarray,
    n_actions: int,
    features_extractor: Optional[PolynomialFeatures] = None,
) -> np.ndarray:
    """
    Retrieve the q-values for a set of observations.
    qf(q_t, action) for all possible actions.

    :param model: Q-value estimator
    :param obs: A batch of observations
    :param n_actions: Number of discrete actions.
    :param features_extractor: Optionally a preprocessor
        to extract features like PolynomialFeatures.
    :return: The predicted q-values for the given observations
        (batch_size, n_actions)
    """
    batch_size = len(obs)
    q_values = np.zeros((batch_size, n_actions))

    ### YOUR CODE HERE

    # Predict q-value for each action
    for action_idx in range(n_actions):
        # Note: we should do one hot encoding if not using CartPole (n_actions > 2)
        # Create a vector of size batch_size for the current action
        actions = action_idx * np.ones((batch_size, 1))
        # Concatenate the observations and the actions to obtain
        # the input to the q-value estimator
        model_input = create_model_input(obs, actions, features_extractor)
        # Predict q-values for the given observation/action combination
        # shape: (batch_size, 1)
        predicted_q_values = model.predict(model_input)
        q_values[:, action_idx] = predicted_q_values

    ### END OF YOUR CODE

    return q_values

In [ ]:
n_observations = 2
n_actions = int(env.action_space.n)

q_values = get_q_values(model, data.observations[:n_observations], n_actions, features_extractor)

assert q_values.shape == (n_observations, n_actions)

### 2. Exercise (5 minutes): write the function to evaluate a model

In [ ]:
def evaluate(
    model: RegressorMixin,
    env: gym.Env,
    n_eval_episodes: int = 10,
    features_extractor: Optional[PolynomialFeatures] = None,
) -> None:
    episode_returns, episode_reward = [], 0.0
    total_episodes = 0
    done = False
    obs, _ = env.reset()
    assert isinstance(env.action_space, spaces.Discrete), "FQI only support discrete actions"

    while total_episodes < n_eval_episodes:
        ### YOUR CODE HERE

        # Retrieve the q-values for the current observation
        q_values = get_q_values(
            model,
            obs[np.newaxis, ...],
            int(env.action_space.n),
            features_extractor,
        )
        # Select the action that maximizes the q-value for each state
        best_action = int(np.argmax(q_values, axis=1).item())

        # Send the action to the env
        obs, reward, terminated, truncated, _ = env.step(best_action)

        ### END OF YOUR CODE

        episode_reward += float(reward)

        done = terminated or truncated
        if done:
            episode_returns.append(episode_reward)
            episode_reward = 0.0
            total_episodes += 1
            obs, _ = env.reset()
    print(f"Total reward = {np.mean(episode_returns):.2f} +/- {np.std(episode_returns):.2f}")

In [ ]:
# Evaluate the first iteration
evaluate(model, env, n_eval_episodes=10, features_extractor=features_extractor)

### 3. Exercise (15 minutes): the fitted Q iterations

In [ ]:
# Max number of iterations
n_iterations = 50
# How often do we evaluate the learned model
eval_freq = 2
# How many episodes to evaluate every eval-freq
n_eval_episodes = 10
# discount factor
gamma = 0.99
# Number of discrete actions
n_actions = int(env.action_space.n)

In [ ]:
for iter_idx in range(n_iterations):
    ### YOUR CODE HERE

    # Construct TD(0) target
    # using current model and the next observations
    next_q_values = get_q_values(
        model,
        data.next_observations,
        n_actions=n_actions,
        features_extractor=features_extractor,
    )
    # Follow-greedy policy: use the action with the highest q-value
    next_q_values = next_q_values.max(axis=1)
    # The new target is the reward + what our agent expect to get
    # if it follows a greedy policy (follow action with the highest q-value)
    targets = data.rewards + gamma * (1 - data.terminateds) * next_q_values
    # Update our q-value estimate with the current target
    model = model_class().fit(current_obs_input, targets)

    ### END OF YOUR CODE

    if (iter_idx + 1) % eval_freq == 0:
        print(f"Iter {iter_idx + 1}")
        print(f"Score: {model.score(current_obs_input, targets):.2f}")
        evaluate(model, env, n_eval_episodes, features_extractor)

### Going further

- play with different models/features_extractor
- play with the discount factor
- play with the number of data collected/used
- combine data from random policy with data from trained model